In [1]:
import warnings; warnings.filterwarnings('ignore')
import time 

import numpy as np
import oxyba as ox
from importlib import reload; reload(ox);

### Generate Random Data

We choose a small $N=30+d$ to amplify the "law of small numbers" effect.

In [2]:
mu_ = -0.05
sd_ =  0.25
N = 32

Fixed seed of 42 for reproducibility.

In [3]:
np.random.seed(42)
y = np.random.normal(loc=mu_, scale=sd_, size=(N,1))

### Prepare the Estimator Function handles

Yes, we could just use `y.mean()` and `y.std()` but we can apply `linreg_ols_lu` to achieve the same with much more obscure lines of codes. 

In [4]:
def func_norm_ols(y):
    import numpy as np
    import oxyba as ox
    beta = ox.linreg_ols_lu(y=y, X=np.ones((len(y),1)))
    yhat = ox.linreg_predict(np.ones((len(y),1)), beta)
    se = np.std(y - yhat)
    return np.array([beta[0][0], se]);
    #return np.array([y.mean(), y.std()])

We can also use `ox.norm_mle` directly but for the sake of consistency we put it into a wrapper

In [5]:
def func_norm_mle(y):
    import oxyba as ox
    mu,sd = ox.norm_mle(y)
    return np.array([mu,sd])

### Delete-1 Jackknife Estimation

In [6]:
d=2

In [7]:
for myfunc in (ox.norm_mle, func_norm_ols):
    #measure time
    start_time = time.time()
    theta_subs, theta_full = ox.jackknife_loop(myfunc, y, d=d);
    proc_time = time.time() - start_time;

    #output some stats
    pvalues, tscores, theta_jack, se_jack, theta_biased = ox.jackknife_stats(theta_subs, theta_full, N=N, d=d);
    ox.jackknife_print(pvalues, tscores, theta_jack, se_jack, theta_biased, theta_full, ['1st Mom','2nd Mom'], N=N, d=d);
    print('{:s} in {:.6f} seconds'.format(myfunc.__name__, proc_time))


Delete-2 Jackknife, N=32, C(N,d)=496
                                  1st Mom    2nd Mom  
                      p-Values:    0.04960    0.00000 
                      t-Scores:   -2.05223    8.64024 
 Jackknife Standard Error (SE):    0.04180    0.02826 
   Jackknife Estimates (theta):   -0.086      0.244   
     Jackknife Biased Estimate:   -0.084      0.232   
          Full Sample Estimate:   -0.084      0.233   
norm_mle in 8.531778 seconds

Delete-2 Jackknife, N=32, C(N,d)=496
                                  1st Mom    2nd Mom  
                      p-Values:    0.05337    0.00000 
                      t-Scores:   -2.01703    8.63729 
 Jackknife Standard Error (SE):    0.04181    0.02826 
   Jackknife Estimates (theta):   -0.084      0.244   
     Jackknife Biased Estimate:   -0.084      0.232   
          Full Sample Estimate:   -0.084      0.233   
func_norm_ols in 0.076866 seconds


The MLE jackknife estimate has a slightly smaller p-value for $\mu$ than the OLS variant. 
However, both still failed to estimate the $\mu$ parameter. 
A p-value around 0.05 is just worthless.